# Lauren Bassett 
# jme9rt 
# DS5001 Final Project  

*The goal of your final project is to apply and integrate what you have learned in this course to
create ​a digital critical edition of a corpus​ that will support exploration of the cultural contents of
that corpus. Cultural content is broadly conceived​—it may be about language use, social events,
cultural categories, sentiments, identity, taste, etc.*

## Part 0: Import Statements/Reading in Files

In [1]:
import pandas as pd 
import numpy as np
%matplotlib inline


xmas_carol_text = 'christmas-carol.txt'
OHCO = ['title', 'chap_num', 'para_num', 'sent_num', 'token_num']

## Part 1: Convert

In [2]:
xmas = open(xmas_carol_text, 'r', encoding='utf-8-sig').readlines()
df = pd.DataFrame(xmas, columns=['line_str'])
df.index.name = 'line_num'
df.line_str = df.line_str.str.strip()
title = df.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')
df['title'] = title

In [3]:
a = df.line_str.str.match(r"\*\*\*\s*START OF (THE|THIS) PROJECT")
b = df.line_str.str.match(r"\*\*\*\s*END OF (THE|THIS) PROJECT")
an = df.loc[a].index[0]
bn = df.loc[b].index[0]
df = df.loc[an + 1 : bn - 2]

What is a Stave? 


"This is an unusual structure that mimics the way a musical piece is put together. The Staves follow the action of the story with the first stave setting the scene, the middle stave showing the turning point for Scrooge and the final stave concluding the story by presenting him as a changed man." -https://www.bbc.co.uk/bitesize/guides/zqxp4j6/revision/2


In [4]:
chap_lines = df.line_str.str.match(r"^\s*(STAVE )(ONE|TWO|THREE|FOUR|FIVE)(?!—)", case=False)
df.loc[chap_lines]

#Remove everything before STAVE ONE
chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]
df.loc[chap_lines, 'chap_num'] = chap_nums
df.chap_num = df.chap_num.ffill()
#Remove all lines containing [Illustration]


In [13]:
df = df.loc[~df.chap_num.isna()] # Remove chapter heading lines
df = df.loc[~chap_lines] # Remove everything before Chapter 1
df.chap_num = df.chap_num.astype('int') # Convert chap_num from float to int

illustration_phrase =  df.line_str.str.match(r'\[Illustration:?.*| ?.*\]') #Remove all lines containing [Illustration]
#df = df.loc[~illustration_phrase]

error: nothing to repeat at position 19

In [ ]:
df.loc[illustration_phrase]

,line_str,title,chap_num
line_num,,,
222,[Illustration],"A Christmas Carol, by Charles Dickens",1
452,"[Illustration: THEY WERE PORTLY GENTLEMEN, PLE...","A Christmas Carol, by Charles Dickens",1
570,[Illustration: _Bob Cratchit went down a slide...,"A Christmas Carol, by Charles Dickens",1
669,[Illustration: _Nobody under the bed; nobody i...,"A Christmas Carol, by Charles Dickens",1
870,[Illustration: ON THE WINGS OF THE WIND],"A Christmas Carol, by Charles Dickens",1
884,[Illustration],"A Christmas Carol, by Charles Dickens",1
971,[Illustration: _The air was filled with phanto...,"A Christmas Carol, by Charles Dickens",1
1013,[Illustration],"A Christmas Carol, by Charles Dickens",1
1018,[Illustration],"A Christmas Carol, by Charles Dickens",2


In [10]:
df[df['line_str'] == '[Illustration: _"What do you call this?" said Joe. "Bed-curtains."_]']

,line_str,title,chap_num
line_num,,,
2897,"[Illustration: _""What do you call this?"" said ...","A Christmas Carol, by Charles Dickens",4


In [49]:
df.loc[illustration_phrase]

,line_str,title,chap_num
line_num,,,


In [51]:
dfc = df.groupby(OHCO[:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string

In [52]:
dfp = dfc['line_str'].str.split(r'\n\n+', expand=True).stack()\
    .to_frame().rename(columns={0:'para_str'})

In [53]:
dfp

para_str
title                                 chap_num                                                      
A Christmas Carol, by Charles Dickens 1        0                                                    
                                               1                                      MARLEY'S GHOST
                                               2   Marley was dead, to begin with. There is no do...
                                               3   Mind! I don't mean to say that I know of my ow...
                                               4   Scrooge knew he was dead? Of course he did. Ho...
...                                                                                              ...
                                      5        72  [Illustration: _"Now, I'll tell you what, my f...
                                               73  Scrooge was better than his word. He did it al...
                                               74  He had no further intercourse with Spirits, bu...
                                               75  +---------------------------------------------...
                                               76  End of the Project Gutenberg EBook of A Christ...

[735 rows x 1 columns]

## Part 2: Annotate

## Part 3: Create

## Part 4: Extend

## Part 5: Present